In [1]:
import numpy as np
import pandas as pd

In [2]:
resaleflat1 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
#resaleflat2 = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
resaleflat3 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
resaleflat4 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
resaleflat = pd.concat((resaleflat1,resaleflat3,resaleflat4), axis=0, sort=True)

resaleflat_l = resaleflat.copy()
resaleflatview = resaleflat1.copy()

In [3]:
resaleflat.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [4]:
resaleflat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139788 entries, 0 to 37152
Data columns (total 11 columns):
block                  139788 non-null object
flat_model             139788 non-null object
flat_type              139788 non-null object
floor_area_sqm         139788 non-null float64
lease_commence_date    139788 non-null int64
month                  139788 non-null object
remaining_lease        87585 non-null object
resale_price           139788 non-null float64
storey_range           139788 non-null object
street_name            139788 non-null object
town                   139788 non-null object
dtypes: float64(2), int64(1), object(8)
memory usage: 12.8+ MB


In [5]:
resaleflat.isnull().sum()

#Remaining lease has a lot of null, need to fix

block                      0
flat_model                 0
flat_type                  0
floor_area_sqm             0
lease_commence_date        0
month                      0
remaining_lease        52203
resale_price               0
storey_range               0
street_name                0
town                       0
dtype: int64

# We had to convert categorical to numerical first to see correlations, now we redo the table using onehotencoding

In [6]:
resaleflat_l.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [7]:
#Convert first two char into integer for years left
resaleflat_l['remaining_lease_year'] = resaleflat_l['remaining_lease'].str.slice(stop=2).astype('float')
#Substring/slice the month out
resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease'].str.slice(start=9,stop=12)

replacemonth = {}
replacemonth[''] = 0
replacemonth['0 m'] = 0
resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease_month'].replace(replacemonth)

resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease_month'].astype('float') / 12
resaleflat_l['remaining_lease'] = resaleflat_l['remaining_lease_year'] + resaleflat_l['remaining_lease_month']
resaleflat_l = resaleflat_l.drop(['remaining_lease_year','remaining_lease_month'], axis=1)

resaleflat_l.month = resaleflat_l.month.apply(lambda x: float(x[0:4])+float(x[5:7])/12)
#Inpute missing values with remaining lease (take current year-month minus the lease start year)
resaleflat_l.remaining_lease[resaleflat_l.remaining_lease.isnull()] = resaleflat_l['month'] - resaleflat_l['lease_commence_date']

#Normalize the month, floor area and remaining lease (timeframe)
resaleflat_l.month = (resaleflat_l.month - resaleflat_l.month.mean()) / resaleflat_l.month.std()
resaleflat_l.floor_area_sqm = (resaleflat_l.floor_area_sqm - resaleflat_l.floor_area_sqm.mean()) / resaleflat_l.floor_area_sqm.std()
resaleflat_l.remaining_lease = (resaleflat_l.remaining_lease - resaleflat_l.remaining_lease.mean()) / resaleflat_l.remaining_lease.std()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#Blocks in Singapore is rather irregular and does not impact on the pricing
#Lease commence date is highly correlated with remaining lease as remaining lease = current date - lease commence date

#Use town instead of street_name for random forest, both are highly correlated so just use one

resaleflat_l = resaleflat_l.drop(columns=['block','lease_commence_date','street_name','flat_type'])

In [9]:
resaleflat_l.head()

,flat_model,floor_area_sqm,month,remaining_lease,resale_price,storey_range,town
0,Improved,-2.163608,0.540959,0.710285,232000.0,10 TO 12,ANG MO KIO
1,New Generation,-1.225543,0.540959,0.681966,250000.0,01 TO 03,ANG MO KIO
2,New Generation,-1.225543,0.540959,0.751191,262000.0,01 TO 03,ANG MO KIO
3,New Generation,-1.184757,0.540959,0.738605,265000.0,04 TO 06,ANG MO KIO
4,New Generation,-1.225543,0.540959,0.751191,265000.0,01 TO 03,ANG MO KIO


In [10]:
all_flat_model = pd.get_dummies(resaleflat_l['flat_model'], prefix='flt')
all_towns = pd.get_dummies(resaleflat_l['town'], prefix='st')

resaleflat_l = resaleflat_l.drop(columns=['flat_model','town'])
resaleflat_l = pd.concat((resaleflat_l,all_flat_model,all_towns), axis=1)

In [11]:
resaleflat_l.storey_range.unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45', '06 TO 10', '01 TO 05', '11 TO 15',
       '16 TO 20', '21 TO 25', '26 TO 30', '36 TO 40', '31 TO 35'],
      dtype=object)

In [12]:
#Get average of storey range e.g. 01 to 03 would be 2
resaleflat_l.storey_range = resaleflat_l.storey_range.apply(lambda x: (float(x[0:2]) + float(x[7:9]) ) /2 )

resaleflat_l.storey_range = (resaleflat_l.storey_range - resaleflat_l.storey_range.mean()) / resaleflat_l.storey_range.std()

In [13]:
storey_resales = resaleflat_l[['storey_range','resale_price']]
storey_resales.corr()

#Storey range has correlation of 0.30 with resale price

,storey_range,resale_price
storey_range,1.000000,0.303071
resale_price,0.303071,1.000000


In [14]:
lease_resales = resaleflat_l[['remaining_lease','resale_price']]
lease_resales.corr()

#Remaining lease only has -0.067 correlation with resale price

,remaining_lease,resale_price
remaining_lease,1.0000,-0.0671
resale_price,-0.0671,1.0000


In [15]:
month_resales = resaleflat_l[['month','resale_price']]
month_resales.corr()

#Year/month bought (month) has -0.07 correlation with resale price

,month,resale_price
month,1.000000,-0.073186
resale_price,-0.073186,1.000000


In [16]:
floor_resales = resaleflat_l[['floor_area_sqm','resale_price']]
floor_resales.corr()

#Floor area SQM has 0.667 correlation with resale price, very high!

,floor_area_sqm,resale_price
floor_area_sqm,1.000000,0.667404
resale_price,0.667404,1.000000


In [17]:
resaleflat_l.head()

,floor_area_sqm,month,remaining_lease,resale_price,storey_range,flt_2-room,flt_Adjoined flat,flt_Apartment,flt_DBSS,flt_Improved,...,st_PASIR RIS,st_PUNGGOL,st_QUEENSTOWN,st_SEMBAWANG,st_SENGKANG,st_SERANGOON,st_TAMPINES,st_TOA PAYOH,st_WOODLANDS,st_YISHUN
0,-2.163608,0.540959,0.710285,232000.0,-0.030180,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,-1.225543,0.540959,0.681966,250000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.225543,0.540959,0.751191,262000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-1.184757,0.540959,0.738605,265000.0,-0.365163,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1.225543,0.540959,0.751191,265000.0,-1.370113,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
resaleflat_l.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139788 entries, 0 to 37152
Data columns (total 52 columns):
floor_area_sqm                139788 non-null float64
month                         139788 non-null float64
remaining_lease               139788 non-null float64
resale_price                  139788 non-null float64
storey_range                  139788 non-null float64
flt_2-room                    139788 non-null uint8
flt_Adjoined flat             139788 non-null uint8
flt_Apartment                 139788 non-null uint8
flt_DBSS                      139788 non-null uint8
flt_Improved                  139788 non-null uint8
flt_Improved-Maisonette       139788 non-null uint8
flt_Maisonette                139788 non-null uint8
flt_Model A                   139788 non-null uint8
flt_Model A-Maisonette        139788 non-null uint8
flt_Model A2                  139788 non-null uint8
flt_Multi Generation          139788 non-null uint8
flt_New Generation            139788 non-null ui

In [19]:
resaleflat_l.sum().index.values

array(['floor_area_sqm', 'month', 'remaining_lease', 'resale_price',
       'storey_range', 'flt_2-room', 'flt_Adjoined flat', 'flt_Apartment',
       'flt_DBSS', 'flt_Improved', 'flt_Improved-Maisonette',
       'flt_Maisonette', 'flt_Model A', 'flt_Model A-Maisonette',
       'flt_Model A2', 'flt_Multi Generation', 'flt_New Generation',
       'flt_Premium Apartment', 'flt_Premium Apartment Loft',
       'flt_Premium Apartment.', 'flt_Premium Maisonette',
       'flt_Simplified', 'flt_Standard', 'flt_Terrace', 'flt_Type S1',
       'flt_Type S2', 'st_ANG MO KIO', 'st_BEDOK', 'st_BISHAN',
       'st_BUKIT BATOK', 'st_BUKIT MERAH', 'st_BUKIT PANJANG',
       'st_BUKIT TIMAH', 'st_CENTRAL AREA', 'st_CHOA CHU KANG',
       'st_CLEMENTI', 'st_GEYLANG', 'st_HOUGANG', 'st_JURONG EAST',
       'st_JURONG WEST', 'st_KALLANG/WHAMPOA', 'st_MARINE PARADE',
       'st_PASIR RIS', 'st_PUNGGOL', 'st_QUEENSTOWN', 'st_SEMBAWANG',
       'st_SENGKANG', 'st_SERANGOON', 'st_TAMPINES', 'st_TOA PAYOH',
  

In [20]:
#from sklearn import svm
#from sklearn.svm import LinearSVR
from sklearn import svm
#there is also just svm or svc (with variable kernel and c) from sklearn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [21]:
#Seperate datasets

X = resaleflat_l.drop('resale_price', axis=1)
Y = resaleflat_l['resale_price']

print(X.shape)
print(Y.shape)

(139788, 51)
(139788,)


In [22]:
#Since we use town names instead of street names, features not that numerous to be selected, just use all
#We also know that each town has its own house pricing so its good to keep all of them

#Choose top 300 features for model out of 561

# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_classif

# X = SelectKBest(f_classif, k=300).fit_transform(X, Y)

In [23]:
print(X.shape)

(139788, 51)


In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Support Vector Regressor

In [25]:
SVR = svm.SVR(kernel='linear',cache_size=1000) #also try: RBF, poly, degree 3

SVR.fit(x_train,y_train)
pred_SVR = SVR.predict(x_test)

In [26]:
pred_SVR

array([383736.26242248, 461360.21975203, 408221.47172374, ...,
       328120.18185634, 405244.44950459, 385978.35334343])

In [27]:
y_test.head()

1040     308000.0
913      460000.0
13013    381000.0
40489    750000.0
10215    480000.0
Name: resale_price, dtype: float64

In [28]:
(y_test - pred_SVR).head()

1040     -75736.262422
913       -1360.219752
13013    -27221.471724
40489    228891.513524
10215     90135.009077
Name: resale_price, dtype: float64

In [33]:
#Let's see how our model performed

import math

math.sqrt(mean_squared_error(y_test, pred_SVR, multioutput='uniform_average'))

109812.10340007194

In [34]:
#Find RMSE for baseline model - i.e. take average house price

base = math.sqrt(((y_test - y_test.mean()) ** 2).mean())

base

#RMSE of $138415 for prediction of house price using average

138415.3257061527

In [35]:
SVR_diff = math.sqrt(((y_test - pred_SVR) ** 2).mean())
SVR_diff

#RMSE of $51868 using linear regression
#RMSE of $37557 using decision tree

109812.10340007201

In [36]:
def RMSE(actual,predict):
    base = math.sqrt(((y_test - y_test.mean()) ** 2).mean())
    x = math.sqrt(((actual - predict) ** 2).mean())
    return x/base

RMSE(y_test, pred_SVR)

0.7933522017149778

In [37]:
#Get R^2 of prediction in linear regression, 0.860
#Get R^2 for random forest, 0.926

SVR.score(x_test,y_test)

0.3705922840339879

In [38]:
y_test.values

array([308000., 460000., 381000., ..., 290000., 310000., 420000.])

In [39]:
pred_SVR

array([383736.26242248, 461360.21975203, 408221.47172374, ...,
       328120.18185634, 405244.44950459, 385978.35334343])

In [40]:
result1 = np.concatenate(([y_test.values], [pred_SVR],[y_test.values - pred_SVR])).T

In [41]:
result1_df = pd.DataFrame(result1, columns=['actual','predict','difference'])

In [42]:
result1_df

,actual,predict,difference
0,308000.0,383736.262422,-75736.262422
1,460000.0,461360.219752,-1360.219752
2,381000.0,408221.471724,-27221.471724
3,750000.0,521108.486476,228891.513524
4,480000.0,389864.990923,90135.009077
5,325000.0,374436.397738,-49436.397738
6,385000.0,383756.531819,1243.468181
7,360000.0,368878.028877,-8878.028877
8,350000.0,414899.544525,-64899.544525
9,337000.0,378341.798689,-41341.798689


In [43]:
print(result1_df['difference'].mean())
print(result1_df['difference'].max())
print(result1_df['difference'].min())

28868.00032044327
650760.6648850718
-181197.553720425


In [44]:
result1_df[result1_df['difference'] > 100000]

,actual,predict,difference
3,750000.0,521108.486476,228891.513524
25,720000.0,452041.518189,267958.481811
26,688888.0,482724.931392,206163.068608
29,640000.0,505376.546086,134623.453914
36,800000.0,433661.392360,366338.607640
45,610000.0,509694.004686,100305.995314
48,638000.0,425336.160484,212663.839516
49,640000.0,494449.224571,145550.775429
68,710000.0,442642.472643,267357.527357
84,568000.0,413642.054498,154357.945502


In [45]:
result1_df[result1_df['difference'] < -100000]

,actual,predict,difference
106,243000.0,349846.024896,-106846.024896
211,320000.0,423615.970955,-103615.970955
314,225000.0,328762.373195,-103762.373195
449,272000.0,372687.386202,-100687.386202
466,310000.0,410229.099618,-100229.099618
470,230000.0,332798.703549,-102798.703549
488,240000.0,350176.790532,-110176.790532
542,266000.0,368591.893980,-102591.893980
612,250000.0,352819.767528,-102819.767528
712,242000.0,346887.156917,-104887.156917


In [46]:
#If we take resale flat from 2000-2012, you can see the model does very poorly with this set

resaleflat_new = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')

resaleflat_new.month = resaleflat_new.month.apply(lambda x: float(x[0:4])+float(x[5:7])/12)
#Inpute missing values with remaining lease (take current year-month minus the lease start year)
resaleflat_new['remaining_lease'] = resaleflat_new['month'] - resaleflat_new['lease_commence_date']

#Normalize the month, floor area and remaining lease (timeframe)
resaleflat_new.month = (resaleflat_new.month - resaleflat_new.month.mean()) / resaleflat_new.month.std()
resaleflat_new.floor_area_sqm = (resaleflat_new.floor_area_sqm - resaleflat_new.floor_area_sqm.mean()) / resaleflat_new.floor_area_sqm.std()
resaleflat_new.remaining_lease = (resaleflat_new.remaining_lease - resaleflat_new.remaining_lease.mean()) / resaleflat_new.remaining_lease.std()

resaleflat_new = resaleflat_new.drop(columns=['block','lease_commence_date','street_name','flat_type'])

all_flat_model = pd.get_dummies(resaleflat_new['flat_model'], prefix='flt')
all_towns = pd.get_dummies(resaleflat_new['town'], prefix='st')

resaleflat_new = resaleflat_new.drop(columns=['flat_model','town'])
resaleflat_new = pd.concat((resaleflat_new,all_flat_model,all_towns), axis=1)

#Get average of storey range e.g. 01 to 03 would be 2
resaleflat_new.storey_range = resaleflat_new.storey_range.apply(lambda x: (float(x[0:2]) + float(x[7:9]) ) /2 )

resaleflat_new.storey_range = (resaleflat_new.storey_range - resaleflat_new.storey_range.mean()) / resaleflat_new.storey_range.std()